In [3]:
import pandas as pd
import mysql.connector


In [4]:
# 1. Extract - Load CSV data into Pandas DataFrames
sales_df = pd.read_csv("sales_data.csv")
product_df = pd.read_csv("product_data.csv")
customer_df = pd.read_csv("customer_data.csv")
date_df = pd.read_csv("date_data.csv")


In [5]:
# 2. Transform - Apply transformations
sales_df["total_sales"] = sales_df["quantity"] * sales_df["price"]  # Transformation-1
date_df["date"] = pd.to_datetime(date_df["date"])  # Transformation-2
date_df["quarter"] = date_df["date"].dt.quarter  # Transformation-3


In [6]:
# 3. Load - Connect to MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="exercises"
)
cursor = mydb.cursor()

In [7]:

# Create DIMENSION TABLES
tables = {
    "dim_product": "product_id INT PRIMARY KEY, product_name VARCHAR(255), category VARCHAR(255)",
    "dim_customer": "customer_id INT PRIMARY KEY, customer_name VARCHAR(255), city VARCHAR(255)",
    "dim_date": "date_id INT PRIMARY KEY, date DATE, month INT, year INT, quarter INT",
    "fact_sales": """transaction_id INT PRIMARY KEY, product_id INT, customer_id INT, 
                     date_id INT, quantity INT, price FLOAT, total_sales FLOAT,
                     FOREIGN KEY (product_id) REFERENCES dim_product(product_id),
                     FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
                     FOREIGN KEY (date_id) REFERENCES dim_date(date_id)"""
}


In [8]:

# Create tables in MySQL
for table, schema in tables.items():
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table} ({schema})")


In [9]:

# Function to insert DataFrame data into MySQL tables
def insert_into_db(df, table_name):
    columns = ", ".join(df.columns)
    values_placeholder = ", ".join(["%s"] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholder})"
    
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))
    mydb.commit()
    print(f"Data inserted into '{table_name}' successfully.")


In [10]:
# Insert transformed data into tables
insert_into_db(product_df, "dim_product")
insert_into_db(customer_df, "dim_customer")
insert_into_db(date_df, "dim_date")
insert_into_db(sales_df, "fact_sales")


Data inserted into 'dim_product' successfully.
Data inserted into 'dim_customer' successfully.
Data inserted into 'dim_date' successfully.
Data inserted into 'fact_sales' successfully.


In [11]:


# Close connection
cursor.close()
mydb.close()
print("ETL process completed. Data loaded into MySQL successfully.")


ETL process completed. Data loaded into MySQL successfully.
